# Laplace Causal Discovery Example\n\nThis notebook demonstrates causal network discovery using the **Laplace** information method with heavy-tailed continuous data.\n\n## Overview\n- Generate synthetic time series with known causal structure\n- Visualize the dynamics and network structure\n- Apply causal discovery using Laplace conditional mutual information\n- Evaluate performance using ROC-AUC metric\n\n

In [ ]:
import numpy as np\nimport matplotlib.pyplot as plt\nimport networkx as nx\nimport seaborn as sns\nfrom sklearn.metrics import roc_auc_score, roc_curve\nimport warnings\nwarnings.filterwarnings('ignore')\n\n# Import causal discovery components\nfrom causalentropy.core.discovery import discover_network\nfrom causalentropy.datasets.synthetic import laplace_coupled_oscillators\n\n# Set plotting style\nplt.style.use('seaborn-v0_8')\nsns.set_palette('coolwarm')\n\nprint('Libraries imported successfully!')

## 1. Create Ground Truth Network\n\nWe'll create a directed graph that represents the true causal relationships.

In [ ]:
# Create ground truth network\nn_nodes = 5\nseed = 42\n\nnp.random.seed(seed)\nG_true = nx.DiGraph()\nG_true.add_nodes_from(range(n_nodes))\n\n# Add specific causal edges\nedges = [(0, 1), (0, 2), (1, 3), (2, 4), (3, 4)]\nG_true.add_edges_from(edges)\n\nprint(f'Ground truth network has {G_true.number_of_nodes()} nodes and {G_true.number_of_edges()} edges')\nprint(f'Edges: {list(G_true.edges())}')\n\nA_true = nx.adjacency_matrix(G_true).toarray()\nprint(f'\\nGround truth adjacency matrix:')\nprint(A_true)

## 2. Generate Synthetic Laplace Data\n\nGenerate heavy-tailed continuous data with the known causal structure.

In [ ]:
# Generate synthetic Laplace data\nT = 200  # Time series length\n\n# Generate data using laplace_coupled_oscillators\ndata, A_generated = laplace_coupled_oscillators(\n    n=n_nodes,\n    T=T,\n    G=G_true,\n    scale_base=1.0, coupling_strength=0.3,\n    seed=seed\n)\n\nprint(f'Generated Laplace data with shape: {data.shape}')\nprint(f'Data statistics:')\nprint(f'  Mean: {np.mean(data):.3f}')\nprint(f'  Std:  {np.std(data):.3f}')\nprint(f'  Range: [{np.min(data):.3f}, {np.max(data):.3f}]')\nprint(f'  Data type: Continuous with heavy tails')

## 3. Visualize Time Series Data\n\nPlot the heavy-tailed continuous data to understand the data characteristics.

In [ ]:
# Plot time series\nfig, axes = plt.subplots(n_nodes, 1, figsize=(12, 8), sharex=True)\nfig.suptitle('Laplace Coupled Oscillators', fontsize=16, fontweight='bold')\n\ntime = np.arange(T)\ncolors = sns.color_palette('coolwarm', n_nodes)\n\nfor i in range(n_nodes):\n    axes[i].plot(time, data[:, i], color=colors[i], alpha=0.8, linewidth=1.5)\n    axes[i].set_ylabel(f'X{i}', fontweight='bold')\n    axes[i].grid(True, alpha=0.3)\n    \n    mean_val = np.mean(data[:, i])\n    axes[i].axhline(mean_val, color='red', linestyle='--', alpha=0.5, \n                   label=f'Mean: {mean_val:.2f}')\n    axes[i].legend(fontsize=8)\n\naxes[-1].set_xlabel('Time', fontweight='bold')\nplt.tight_layout()\nplt.show()

## 4. Visualize Ground Truth Network

In [ ]:
# Plot ground truth network\nplt.figure(figsize=(10, 8))\n\npos = nx.spring_layout(G_true, seed=seed, k=2, iterations=50)\n\nnx.draw_networkx_nodes(G_true, pos, node_color='lightsalmon', \n                       node_size=1500, alpha=0.8)\nnx.draw_networkx_edges(G_true, pos, edge_color='darkred', \n                       arrows=True, arrowsize=20, width=2, alpha=0.7)\nnx.draw_networkx_labels(G_true, pos, {i: f'X{i}' for i in range(n_nodes)},\n                        font_size=14, font_weight='bold')\n\nplt.title('Ground Truth Causal Network\\n(Laplace Data)', \n          fontsize=16, fontweight='bold')\nplt.axis('off')\nplt.tight_layout()\nplt.show()

## 5. Apply Causal Discovery\n\nUse the Laplace method to discover causal relationships.

In [ ]:
# Apply causal discovery with Laplace method\nprint('Applying causal discovery with Laplace information method...')\nprint('This may take a few moments...\\n')\n\nmethods_to_test = ['standard', 'alternative']\ndiscovered_networks = {}\n\nfor method in methods_to_test:\n    print(f'Running {method} method...')\n    \n    G_discovered = discover_network(\n        data=data,\n        method=method,\n        information='laplace',\n        max_lag=2,\n        alpha_forward=0.05,\n        alpha_backward=0.05,\n        n_shuffles=100\n    )\n    \n    discovered_networks[method] = G_discovered\n    print(f'  Discovered {G_discovered.number_of_edges()} edges')\n    print(f'  Edges: {list(G_discovered.edges())}\\n')

## 6. Calculate ROC-AUC Performance

In [ ]:
def calculate_roc_auc(true_adj, discovered_graph):\n    \"\"\"Calculate ROC-AUC for network discovery performance.\"\"\"\n    n = true_adj.shape[0]\n    \n    G_int = nx.DiGraph()\n    G_int.add_nodes_from(range(n))\n    for edge in discovered_graph.edges():\n        src = int(edge[0].replace('X', '')) if 'X' in str(edge[0]) else int(edge[0])\n        dst = int(edge[1].replace('X', '')) if 'X' in str(edge[1]) else int(edge[1])\n        G_int.add_edge(src, dst)\n    \n    discovered_adj = nx.adjacency_matrix(G_int, nodelist=range(n)).toarray()\n    \n    mask = ~np.eye(n, dtype=bool).flatten()\n    y_true = true_adj.flatten()[mask]\n    y_scores = discovered_adj.flatten()[mask]\n    \n    if len(np.unique(y_true)) > 1:\n        auc_score = roc_auc_score(y_true, y_scores)\n        fpr, tpr, _ = roc_curve(y_true, y_scores)\n        return auc_score, fpr, tpr\n    else:\n        return None, None, None\n\n# Calculate ROC-AUC\nresults = {}\nplt.figure(figsize=(10, 6))\n\nfor method, G_disc in discovered_networks.items():\n    auc_score, fpr, tpr = calculate_roc_auc(A_true, G_disc)\n    \n    if auc_score is not None:\n        results[method] = {'auc': auc_score, 'fpr': fpr, 'tpr': tpr}\n        plt.plot(fpr, tpr, linewidth=2, label=f'{method} (AUC = {auc_score:.3f})')\n        print(f'{method} method: ROC-AUC = {auc_score:.3f}')\n\nplt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Random (AUC = 0.500)')\nplt.xlabel('False Positive Rate', fontweight='bold')\nplt.ylabel('True Positive Rate', fontweight='bold')\nplt.title('ROC Curves for Laplace Causal Discovery', fontweight='bold')\nplt.legend()\nplt.grid(True, alpha=0.3)\nplt.tight_layout()\nplt.show()

## 7. Conclusions

In [ ]:
print('\\n' + '='*60)\nprint('EXPERIMENT CONCLUSIONS - LAPLACE CAUSAL DISCOVERY')\nprint('='*60)\n\nprint(f'📊 DATA CHARACTERISTICS:')\nprint(f'  • Data type: Continuous with heavy tails')\nprint(f'  • Time series length: {T}')\nprint(f'  • Number of variables: {n_nodes}')\nprint(f'  • Ground truth edges: {G_true.number_of_edges()}')\n\nprint(f'🔍 DISCOVERY RESULTS:')\nbest_auc = 0\nbest_method = None\nfor method, G_disc in discovered_networks.items():\n    auc_val = results.get(method, {}).get('auc', 0)\n    print(f'  • {method.capitalize()}: {G_disc.number_of_edges()} edges, AUC = {auc_val:.3f}')\n    if auc_val > best_auc:\n        best_auc = auc_val\n        best_method = method\n\nif best_method:\n    print(f'🏆 BEST METHOD: {best_method.upper()}')\n    print(f'  • ROC-AUC: {best_auc:.3f}')\n\nprint(f'💡 LAPLACE METHOD INSIGHTS:')\nprint(f'  • Designed for heavy-tailed continuous data')\nprint(f'  • Data type: Continuous with heavy tails')\nprint(f'  • Performance depends on data characteristics and coupling strength')\n\nprint('Experiment completed! 🎉')